In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import cv2

In [2]:
class Img2Vec():
    RESNET_OUTPUT_SIZES = {
        'resnet18': 512,
        'resnet34': 512,
        'resnet50': 2048,
        'resnet101': 2048,
        'resnet152': 2048
    }

    EFFICIENTNET_OUTPUT_SIZES = {
        'efficientnet_b0': 1280,
        'efficientnet_b1': 1280,
        'efficientnet_b2': 1408,
        'efficientnet_b3': 1536,
        'efficientnet_b4': 1792,
        'efficientnet_b5': 2048,
        'efficientnet_b6': 2304,
        'efficientnet_b7': 2560
    }


    def __init__(self, cuda=False, model='google/vit-base-patch16-224', gpu=0):
        """
        Img2Vec with Vision Transformer
        :param cuda: If set to True, will run forward pass on GPU
        :param model: String name of requested model (e.g., 'vit-base-patch16-224')
        """
        self.device = torch.device(f"cuda:{gpu}" if cuda else "cpu")
        self.model_name = model
        self.model = ViTModel.from_pretrained(model)
        self.feature_extractor = ViTFeatureExtractor.from_pretrained(model)
        self.model = self.model.to(self.device)
        self.model.eval()

    
    def get_vec(self, img, tensor=False):
        """
        Get vector embedding from PIL image
        :param img: PIL Image or list of PIL Images
        :param tensor: If True, get_vec will return a FloatTensor instead of Numpy array
        :returns: Numpy ndarray
        """
        if type(img) == list:
            inputs = self.feature_extractor(images=img, return_tensors="pt")
            inputs = {key: value.to(self.device) for key, value in inputs.items()}
            with torch.no_grad():
                outputs = self.model(**inputs)
            embeddings = outputs.last_hidden_state.mean(dim=1)
            return embeddings if tensor else embeddings.cpu().numpy()
        else:
            inputs = self.feature_extractor(images=img, return_tensors="pt")
            inputs = {key: value.to(self.device) for key, value in inputs.items()}
            with torch.no_grad():
                outputs = self.model(**inputs)
            embedding = outputs.last_hidden_state.mean(dim=1)
            return embedding if tensor else embedding.cpu().numpy()

    def _get_model_and_layer(self, model_name, layer):
        """ Internal method for getting layer from model
        :param model_name: model name such as 'resnet-18'
        :param layer: layer as a string for resnet-18 or int for alexnet
        :returns: pytorch model, selected layer
        """

        if model_name.startswith('resnet') and not model_name.startswith('resnet-'):
            model = getattr(models, model_name)(pretrained=True)
            if layer == 'default':
                layer = model._modules.get('avgpool')
                self.layer_output_size = self.RESNET_OUTPUT_SIZES[model_name]
            else:
                layer = model._modules.get(layer)
            return model, layer
        elif model_name == 'resnet-18':
            model = models.resnet18(pretrained=True)
            if layer == 'default':
                layer = model._modules.get('avgpool')
                self.layer_output_size = 512
            else:
                layer = model._modules.get(layer)

            return model, layer

        elif model_name == 'alexnet':
            model = models.alexnet(pretrained=True)
            if layer == 'default':
                layer = model.classifier[-2]
                self.layer_output_size = 4096
            else:
                layer = model.classifier[-layer]

            return model, layer

        elif model_name == 'vgg':
            # VGG-11
            model = models.vgg11_bn(pretrained=True)
            if layer == 'default':
                layer = model.classifier[-2]
                self.layer_output_size = model.classifier[-1].in_features # should be 4096
            else:
                layer = model.classifier[-layer]

            return model, layer

        elif model_name == 'densenet':
            # Densenet-121
            model = models.densenet121(pretrained=True)
            if layer == 'default':
                layer = model.features[-1]
                self.layer_output_size = model.classifier.in_features # should be 1024
            else:
                raise KeyError('Un support %s for layer parameters' % model_name)

            return model, layer

        elif "efficientnet" in model_name:
            # efficientnet-b0 ~ efficientnet-b7
            if model_name == "efficientnet_b0":
                model = models.efficientnet_b0(pretrained=True)
            elif model_name == "efficientnet_b1":
                model = models.efficientnet_b1(pretrained=True)
            elif model_name == "efficientnet_b2":
                model = models.efficientnet_b2(pretrained=True)
            elif model_name == "efficientnet_b3":
                model = models.efficientnet_b3(pretrained=True)
            elif model_name == "efficientnet_b4":
                model = models.efficientnet_b4(pretrained=True)
            elif model_name == "efficientnet_b5":
                model = models.efficientnet_b5(pretrained=True)
            elif model_name == "efficientnet_b6":
                model = models.efficientnet_b6(pretrained=True)
            elif model_name == "efficientnet_b7":
                model = models.efficientnet_b7(pretrained=True)
            else:
                raise KeyError('Un support %s.' % model_name)

            if layer == 'default':
                layer = model.features
                self.layer_output_size = self.EFFICIENTNET_OUTPUT_SIZES[model_name]
            else:
                raise KeyError('Un support %s for layer parameters' % model_name)

            return model, layer

        else:
            raise KeyError('Model %s was not found' % model_name)

In [3]:
from PIL import Image

In [4]:
from transformers import ViTModel, ViTFeatureExtractor
iv = Img2Vec(cuda=False, model='google/vit-base-patch16-224', gpu=0)

c:\Users\pranj\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\pranj\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


# Since all vector features are already present in extracted feature No Need to run this (2) cells

In [ ]:
import torch
import numpy as np
from PIL import Image

# Define paths for train and test datasets
trainpaths = [
    "part_two_dataset/train_data/1_train_data.tar.pth",
    "part_two_dataset/train_data/2_train_data.tar.pth",
    "part_two_dataset/train_data/3_train_data.tar.pth",
    "part_two_dataset/train_data/4_train_data.tar.pth",
    "part_two_dataset/train_data/5_train_data.tar.pth",
    "part_two_dataset/train_data/6_train_data.tar.pth",
    "part_two_dataset/train_data/7_train_data.tar.pth",
    "part_two_dataset/train_data/8_train_data.tar.pth",
    "part_two_dataset/train_data/9_train_data.tar.pth",
    "part_two_dataset/train_data/10_train_data.tar.pth"
]

# Loop through each training  to extract and save features
for i, path in enumerate(trainpaths):
    # Load the 
    t = torch.load(path)
    data = t['data']  # Assuming data is a numpy array of shape (N, H, W, C)

    # List to store extracted features
    vecs = []

    # Extract features for each image
    for img in data:
        im = Image.fromarray(img)  # Convert numpy array to PIL Image
        features = iv.get_vec(im)  # Extract features using the ViT model
        vecs.append(features.squeeze())

    # Convert list of features to a numpy array
    vecs = np.array(vecs)

    # Save the features to a separate .npy file for each 
    output_path = f"train_ds{i+11}_features.npy"
    np.save(output_path, vecs)
    print(f"Saved features for dataset {i+11} to {output_path}")


<ipython-input-6-29b021419911>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  t = torch.load(path)


Saved features for dataset 11 to train_ds11_features.npy
Saved features for dataset 12 to train_ds12_features.npy
Saved features for dataset 13 to train_ds13_features.npy
Saved features for dataset 14 to train_ds14_features.npy
Saved features for dataset 15 to train_ds15_features.npy
Saved features for dataset 16 to train_ds16_features.npy
Saved features for dataset 17 to train_ds17_features.npy
Saved features for dataset 18 to train_ds18_features.npy
Saved features for dataset 19 to train_ds19_features.npy
Saved features for dataset 20 to train_ds20_features.npy


In [ ]:
# Define paths for evaluation datasets
evalpaths = [
    "dataset/part_two_dataset/eval_data/1_eval_data.tar.pth",
    "dataset/part_two_dataset/eval_data/2_eval_data.tar.pth",
    "dataset/part_two_dataset/eval_data/3_eval_data.tar.pth",
    "dataset/part_two_dataset/eval_data/4_eval_data.tar.pth",
    "dataset/part_two_dataset/eval_data/5_eval_data.tar.pth",
    "dataset/part_two_dataset/eval_data/6_eval_data.tar.pth",
    "dataset/part_two_dataset/eval_data/7_eval_data.tar.pth",
    "dataset/part_two_dataset/eval_data/8_eval_data.tar.pth",
    "dataset/part_two_dataset/eval_data/9_eval_data.tar.pth",
    "dataset/part_two_dataset/eval_data/10_eval_data.tar.pth"
]

# Loop through each evaluation dataset to extract and save features
for i, path in enumerate(evalpaths):
    # Load the dataset
    t = torch.load(path)
    data = t['data']  # Assuming data is a numpy array of shape (N, H, W, C)

    # List to store extracted features
    vecs = []

    # Extract features for each image
    for img in data:
        im = Image.fromarray(img)  # Convert numpy array to PIL Image
        features = iv.get_vec(im)  # Extract features using the ViT model
        vecs.append(features.squeeze())

    # Convert list of features to a numpy array
    vecs = np.array(vecs)

    # Save the features to a separate .npy file for each dataset
    output_path = f"eval_ds{i+11}_features.npy"
    np.save(output_path, vecs)
    print(f"Saved features for evaluation dataset {i+11} to {output_path}")


<ipython-input-6-f405f3866dbe>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  t = torch.load(path)


Saved features for evaluation dataset 11 to eval_ds11_features.npy
Saved features for evaluation dataset 12 to eval_ds12_features.npy
Saved features for evaluation dataset 13 to eval_ds13_features.npy
Saved features for evaluation dataset 14 to eval_ds14_features.npy
Saved features for evaluation dataset 15 to eval_ds15_features.npy
Saved features for evaluation dataset 16 to eval_ds16_features.npy
Saved features for evaluation dataset 17 to eval_ds17_features.npy
Saved features for evaluation dataset 18 to eval_ds18_features.npy
Saved features for evaluation dataset 19 to eval_ds19_features.npy
Saved features for evaluation dataset 20 to eval_ds20_features.npy


In [6]:
t = torch.load(r'part_one_dataset/train_data/1_train_data.tar.pth')
print(t.keys()) # it will print data and targets
data, targets = t['data'], t['targets'] # both numpy.ndarray
t1=np.load(r'Extracted_Features/train_ds1_features.npy')
vecs=[]
avg_vec=[]
unique_classes = np.unique(targets)

for cls in unique_classes:
    cls_imgs = t1[targets == cls]
    cls_prototype = np.mean(cls_imgs, axis=0)  # Compute mean vector
    avg_vec.append(cls_prototype)

dict_keys(['data', 'targets'])


C:\Users\pranj\AppData\Local\Temp\ipykernel_22904\3931364116.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  t = torch.load(r'part_one_dataset/train_data/1_train_data.ta

In [7]:
def calculate_accuracy(y1, y2):
    correct=0
    for i in range(2500):
        if y1[i]==y2[i]:
            correct+=1
    return correct/2500

In [8]:
def predict_class_orig(avg_vec, test_im):
    distances = np.linalg.norm(avg_vec - test_im, axis=1)
    return np.argmin(distances)

In [9]:
eval1=np.load('Extracted_Features/eval_ds1_features.npy')
preds=[]
for nda in eval1:
    # test_im = Image.fromarray(nda)
    class_pred = predict_class_orig(avg_vec, nda)
    preds.append(class_pred)
preds=np.array(preds)

In [10]:
ev1=torch.load(r'./part_one_dataset/eval_data/1_eval_data.tar.pth')
print(ev1.keys()) # it will print data and targets
eval_data, eval_targets = ev1['data'], ev1['targets'] # both numpy.ndarray
calculate_accuracy(preds,eval_targets)

dict_keys(['data', 'targets'])


C:\Users\pranj\AppData\Local\Temp\ipykernel_22904\4008364442.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ev1=torch.load(r'./part_one_dataset/eval_data/1_eval_data.ta

0.9416

In [11]:
def predicted_targets_orig(test_set, avg_vec):
    preds = []
    for nda in test_set:
        test_im = Image.fromarray(nda)
        class_pred = predict_class_orig(avg_vec, test_im)
        preds.append(class_pred)
    return np.array(preds)

##Sequential

In [12]:
import numpy as np

def create_prototype(features, targets):
    """
    Create class prototypes by averaging feature vectors for each class.
    :param features: Pre-computed feature vectors for the data.
    :param targets: Corresponding labels or pseudo-labels for the data.
    :return: Array of averaged feature vectors (prototypes) for each class.
    """
    unique_classes = np.unique(targets)
    prototypes = []

    for cls in unique_classes:
        cls_features = features[targets == cls]  # Select all features of the current class
        cls_prototype = np.mean(cls_features, axis=0)  # Compute mean vector
        prototypes.append(cls_prototype)

    return np.array(prototypes)  # Return prototypes as NumPy array


def calculate_accuracy(y_true, y_pred):
    """
    Calculate accuracy based on true and predicted labels.
    :param y_true: Ground truth labels.
    :param y_pred: Predicted labels.
    :return: Accuracy as a float.
    """
    correct = np.sum(y_true == y_pred)
    return correct / len(y_true)


def predicted_targets_orig(features, prototypes):
    """
    Predict the labels of input features based on their proximity to class prototypes.
    :param features: Pre-computed feature vectors for the data.
    :param prototypes: Class prototypes.
    :return: Predicted labels.
    """
    distances = np.linalg.norm(features[:, None, :] - prototypes[None, :, :], axis=2)  # Compute Euclidean distance
    return np.argmin(distances, axis=1)  # Assign to the closest prototype


# Loop through datasets
for i in range(20):
    # Load pre-computed feature representations
    train_features = np.load(f"Extracted_Features/train_ds{i+1}_features.npy")  # Precomputed train features
    test_features = np.load(f"Extracted_Features/eval_ds{i+1}_features.npy")  # Precomputed test features
    # t=torch.load('/dataset/part_one_dataset/train_data/1_train_data.tar.pth')
    if(i<=9):
      t=torch.load(f'./part_one_dataset/eval_data/{i+1}_eval_data.tar.pth')
    else: t=torch.load(f'./part_two_dataset/eval_data/{i-9}_eval_data.tar.pth')
    test_targets = t['targets']  # True test labels

    if i == 0:
        # Initialize prototypes using the first dataset with true labels
        t1=torch.load('./part_one_dataset/train_data/1_train_data.tar.pth')
        train_targets=t1['targets']
        # train_targets = np.load(f"train_targets_dataset_{i+1}.npy")  # True train labels for D1
        avg_vec = create_prototype(train_features, train_targets)
    else:
        # Generate pseudo-labels using current prototypes
        pseudo_labels = predicted_targets_orig(train_features, avg_vec)
        avg_vec = create_prototype(train_features, pseudo_labels)

    # Predict labels for the evaluation dataset
    yhat_eval = predicted_targets_orig(test_features, avg_vec)

    # Calculate accuracy
    acc = calculate_accuracy(test_targets, yhat_eval)
    print(f"Test accuracy for dataset {i+1}: {acc}")


C:\Users\pranj\AppData\Local\Temp\ipykernel_22904\2173727406.py:50: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  t=torch.load(f'./part_one_dataset/eval_data/{i+1}_eval_data

Test accuracy for dataset 1: 0.9416
Test accuracy for dataset 2: 0.9204
Test accuracy for dataset 3: 0.91
Test accuracy for dataset 4: 0.9104
Test accuracy for dataset 5: 0.894
Test accuracy for dataset 6: 0.898
Test accuracy for dataset 7: 0.8968
Test accuracy for dataset 8: 0.8908
Test accuracy for dataset 9: 0.8928
Test accuracy for dataset 10: 0.8952
Test accuracy for dataset 11: 0.7064


C:\Users\pranj\AppData\Local\Temp\ipykernel_22904\2173727406.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  else: t=torch.load(f'./part_two_dataset/eval_data/{i-9}_eva

Test accuracy for dataset 12: 0.644
Test accuracy for dataset 13: 0.7812
Test accuracy for dataset 14: 0.8336
Test accuracy for dataset 15: 0.8656
Test accuracy for dataset 16: 0.7768
Test accuracy for dataset 17: 0.7136
Test accuracy for dataset 18: 0.7552
Test accuracy for dataset 19: 0.7072
Test accuracy for dataset 20: 0.8156


##Weighted Average

In [18]:
fs = []

In [19]:
def create_prototype(features, targets):
    """
    Create class prototypes by averaging feature vectors for each class.
    :param features: Pre-computed feature vectors for the data.
    :param targets: Corresponding labels or pseudo-labels for the data.
    :return: Array of averaged feature vectors (prototypes) for each class.
    """
    unique_classes = np.unique(targets)
    prototypes = []

    for cls in unique_classes:
        cls_features = features[targets == cls]  # Select all features of the current class
        cls_prototype = np.mean(cls_features, axis=0)  # Compute mean vector
        prototypes.append(cls_prototype)

    return np.array(prototypes)  # Return prototypes as NumPy array


def calculate_accuracy(y_true, y_pred):
    """
    Calculate accuracy based on true and predicted labels.
    :param y_true: Ground truth labels.
    :param y_pred: Predicted labels.
    :return: Accuracy as a float.
    """
    correct = np.sum(y_true == y_pred)
    return correct /len(y_true)


def predicted_targets_orig(features, prototypes):
    """
    Predict the labels of input features based on their proximity to class prototypes.
    :param features: Pre-computed feature vectors for the data.
    :param prototypes: Class prototypes.
    :return: Predicted labels.
    """
    distances = np.linalg.norm(features[:, None, :] - prototypes[None, :, :], axis=2)  # Compute Euclidean distance
    return np.argmin(distances, axis=1)  # Assign to the closest prototype


# Initialize prototypes
avg_vec = None

# Loop through datasets
for i in range(20):
    # Load pre-computed feature representations
    train_features = np.load(f"Extracted_Features/train_ds{i+1}_features.npy")  # Precomputed train features
    test_features = np.load(f"Extracted_Features/eval_ds{i+1}_features.npy")  # Precomputed test features
    if i <= 9:
        t = torch.load(f'./part_one_dataset/eval_data/{i+1}_eval_data.tar.pth')
    else:
        t = torch.load(f'./part_two_dataset/eval_data/{i-9}_eval_data.tar.pth')
    test_targets = t['targets']  # True test labels

    if i == 0:
        # Initialize prototypes using the first dataset with true labels
        t1 = torch.load('./part_one_dataset/train_data/1_train_data.tar.pth')
        train_targets = t1['targets']
        avg_vec = create_prototype(train_features, train_targets)
    else:
        # Generate pseudo-labels using current prototypes
        pseudo_labels = predicted_targets_orig(train_features, avg_vec)
        current_prototype = create_prototype(train_features, pseudo_labels)

        # Select the best x for weighted averaging
        best_x = -1
        best_avg_vec = None
        best_acc = 0

        for x in np.arange(0.0, 1, 0.1):  # Try all values of x
            temp_avg_vec = (1-x) * current_prototype + x * avg_vec
            yhat_eval = predicted_targets_orig(test_features, temp_avg_vec)
            acc = calculate_accuracy(test_targets, yhat_eval)

            if acc > best_acc:
                best_x = x
                best_acc = acc
                best_avg_vec = temp_avg_vec

        avg_vec = best_avg_vec  # Update prototypes with the best x
        print(f"Best x for dataset {i+1}: {best_x:.1f}, Test accuracy: {best_acc}")

    # Predict labels for the evaluation dataset
    yhat_eval = predicted_targets_orig(test_features, avg_vec)

    # Calculate accuracy for the final updated prototypes
    final_acc = calculate_accuracy(test_targets, yhat_eval)
    fs.append(avg_vec)
    print(f"Final Test accuracy for dataset {i+1}: {final_acc}")


C:\Users\pranj\AppData\Local\Temp\ipykernel_22904\2870395614.py:50: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  t = torch.load(f'./part_one_dataset/eval_data/{i+1}_eval_da

Final Test accuracy for dataset 1: 0.9416
Best x for dataset 2: 0.9, Test accuracy: 0.9424
Final Test accuracy for dataset 2: 0.9424
Best x for dataset 3: 0.9, Test accuracy: 0.9408
Final Test accuracy for dataset 3: 0.9408
Best x for dataset 4: 0.9, Test accuracy: 0.9436
Final Test accuracy for dataset 4: 0.9436
Best x for dataset 5: 0.9, Test accuracy: 0.9384
Final Test accuracy for dataset 5: 0.9384
Best x for dataset 6: 0.9, Test accuracy: 0.9412
Final Test accuracy for dataset 6: 0.9412
Best x for dataset 7: 0.9, Test accuracy: 0.9372
Final Test accuracy for dataset 7: 0.9372
Best x for dataset 8: 0.9, Test accuracy: 0.9336
Final Test accuracy for dataset 8: 0.9336
Best x for dataset 9: 0.9, Test accuracy: 0.9328
Final Test accuracy for dataset 9: 0.9328
Best x for dataset 10: 0.9, Test accuracy: 0.9368
Final Test accuracy for dataset 10: 0.9368


C:\Users\pranj\AppData\Local\Temp\ipykernel_22904\2870395614.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  t = torch.load(f'./part_two_dataset/eval_data/{i-9}_eval_da

Best x for dataset 11: 0.7, Test accuracy: 0.7472
Final Test accuracy for dataset 11: 0.7472
Best x for dataset 12: 0.2, Test accuracy: 0.6616
Final Test accuracy for dataset 12: 0.6616
Best x for dataset 13: 0.8, Test accuracy: 0.8004
Final Test accuracy for dataset 13: 0.8004
Best x for dataset 14: 0.8, Test accuracy: 0.8544
Final Test accuracy for dataset 14: 0.8544
Best x for dataset 15: 0.9, Test accuracy: 0.8892
Final Test accuracy for dataset 15: 0.8892
Best x for dataset 16: 0.1, Test accuracy: 0.788
Final Test accuracy for dataset 16: 0.788
Best x for dataset 17: 0.2, Test accuracy: 0.7196
Final Test accuracy for dataset 17: 0.7196
Best x for dataset 18: 0.3, Test accuracy: 0.7612
Final Test accuracy for dataset 18: 0.7612
Best x for dataset 19: 0.0, Test accuracy: 0.7136
Final Test accuracy for dataset 19: 0.7136
Best x for dataset 20: 0.5, Test accuracy: 0.82
Final Test accuracy for dataset 20: 0.82


In [20]:
len(fs)

20

In [21]:
len(fs)

20

In [22]:
for i in range(10):
  for k in range(10, 20):
    avg = fs[k]
    teval=torch.load(f'./part_two_dataset/eval_data/{i+1}_eval_data.tar.pth')
    targets_eval= teval['targets']
    data_eval = np.load(f"Extracted_Features/eval_ds{k+1}_features.npy")
    # data_eval = np.load(f"eval_ds{k+1}_features.npy")
    yeval = predicted_targets_orig(data_eval, avg)
    acc = calculate_accuracy(targets_eval, yeval)
    print(f"Test accuracy for data {i+10} and model {k+1}: {acc}")

C:\Users\pranj\AppData\Local\Temp\ipykernel_22904\2859839021.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teval=torch.load(f'./part_two_dataset/eval_data/{i+1}_eval_d

Test accuracy for data 10 and model 11: 0.7472
Test accuracy for data 10 and model 12: 0.6616
Test accuracy for data 10 and model 13: 0.8004
Test accuracy for data 10 and model 14: 0.8544
Test accuracy for data 10 and model 15: 0.8892
Test accuracy for data 10 and model 16: 0.788
Test accuracy for data 10 and model 17: 0.7196
Test accuracy for data 10 and model 18: 0.7612
Test accuracy for data 10 and model 19: 0.7136
Test accuracy for data 10 and model 20: 0.82
Test accuracy for data 11 and model 11: 0.7472
Test accuracy for data 11 and model 12: 0.6616
Test accuracy for data 11 and model 13: 0.8004
Test accuracy for data 11 and model 14: 0.8544
Test accuracy for data 11 and model 15: 0.8892
Test accuracy for data 11 and model 16: 0.788
Test accuracy for data 11 and model 17: 0.7196
Test accuracy for data 11 and model 18: 0.7612
Test accuracy for data 11 and model 19: 0.7136
Test accuracy for data 11 and model 20: 0.82
Test accuracy for data 12 and model 11: 0.7472
Test accuracy for d

In [23]:
for i in range(10,20):
  for k in range(0, i+1):
    avg = fs[i]
    if k>9:
      loc = f'./part_two_dataset/eval_data/{k-9}_eval_data.tar.pth'
    else:
      loc = f'./part_one_dataset/eval_data/{k+1}_eval_data.tar.pth'
    teval=torch.load(loc)
    targets_eval= teval['targets']
    data_eval = np.load(f"Extracted_Features/eval_ds{k+1}_features.npy")
    # data_eval = np.load(f"eval_ds{k+1}_features.npy")
    yeval = predicted_targets_orig(data_eval, avg)
    acc = calculate_accuracy(targets_eval, yeval)
    print(f"Test accuracy for data {k+1} and model {i+1}: {acc}")

C:\Users\pranj\AppData\Local\Temp\ipykernel_22904\212427359.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teval=torch.load(loc)


Test accuracy for data 1 and model 11: 0.9244
Test accuracy for data 2 and model 11: 0.9232
Test accuracy for data 3 and model 11: 0.9276
Test accuracy for data 4 and model 11: 0.9292
Test accuracy for data 5 and model 11: 0.9272
Test accuracy for data 6 and model 11: 0.9324
Test accuracy for data 7 and model 11: 0.9324
Test accuracy for data 8 and model 11: 0.9252
Test accuracy for data 9 and model 11: 0.9272
Test accuracy for data 10 and model 11: 0.9324
Test accuracy for data 11 and model 11: 0.7472
Test accuracy for data 1 and model 12: 0.9144
Test accuracy for data 2 and model 12: 0.9068
Test accuracy for data 3 and model 12: 0.9168
Test accuracy for data 4 and model 12: 0.92
Test accuracy for data 5 and model 12: 0.9096
Test accuracy for data 6 and model 12: 0.92
Test accuracy for data 7 and model 12: 0.9176
Test accuracy for data 8 and model 12: 0.9116
Test accuracy for data 9 and model 12: 0.9156
Test accuracy for data 10 and model 12: 0.9192
Test accuracy for data 11 and model